In [1]:
import numpy as np
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import pickle
from stochastic_cramer_rao import cramer_rao

# Number of sensors
M = 12

# Number of nodes
K = 6

# Locations of the nodes with respect to initial node
locations = (np.array([0,0]),np.array([0.45,0.99]),np.array([3.02,0.45]),np.array([5.61,0.90])
             ,np.array([8.03,1.46]),np.array([8.70,0.50]))

# Direction of arrivals
angles = np.array([5,-14,-10])*2*(np.pi)/360
K_l = np.array([np.sin(angles), np.cos(angles)])

# Steering vector is implemented. It should have the shape: (12,3)
for j in range(0,len(locations)):
    for i in range(0,len(angles)):
        first_part = np.exp(1j*np.pi*locations[j].dot(K_l[:,i]))
        second_part = np.array([1, np.exp(1j*np.pi*1*np.sin(angles[i]))])
        A_k = np.transpose(np.array(first_part*second_part, ndmin=2))
        if i == 0:
            B_k = A_k
        else:
            B_k = np.concatenate((B_k, A_k), axis=1)
        
        if i == len(angles)-1:
            if j == 0:
                A = np.transpose(B_k)
            else:
                A = np.concatenate((A, np.transpose(B_k)), axis=1)

A = np.transpose(A)

# uncomment here and comment for N_samples if you want to iterate over SNR
for x in range(0, 1):
    #MSE = np.zeros(1000)
    MSE = np.zeros(80)
    #cramer = np.zeros(1000)
    #cramer = np.zeros(80)
    for snr_dB in range(-20,60):
    #for N_samples in range(2,1002):
        
        for i in range(500):

            #snr_dB = 10
            # Signal(A*s) to noise(n) ratio
            received_snr = 10**(snr_dB/20)
            ratio_As_to_s = 1/3
            snr = received_snr*ratio_As_to_s
            N_samples = 100
            #snr = 10**(snr_dB/20)

            # Source signal implementation (shape: (3,500))
            signal = np.random.normal(0,np.sqrt(snr),(3,N_samples))
            #w = np.atleast_2d([np.pi/3, np.pi/4, np.pi/5]).T
            #signal = (np.sqrt(snr/3))*np.exp(1j*w*(np.atleast_2d(np.arange(1,N_samples+1))))

            # Received signal power on sensors
            signal_power = sum(sum(np.abs(A.dot(signal))**2))

            # Noise signal implementation (shape: (12,500))
            noise = np.random.normal(0,np.sqrt(0.5),(12,N_samples)) + 1j*np.random.normal(0,np.sqrt(0.5),(12,N_samples))
            noise_power  = sum(sum(np.abs(noise)**2))
            if i == 0:
                print("SIGNAL TO NOISE RATIO")
                print(signal_power/noise_power)

            # Received signal (shape: (12,500))
            z = A.dot(signal) + noise

            # Sample covariance matrix
            R_sample = z.dot(z.conj().T)

            # Eigenvalue and eigenvectors
            w_sample, v_sample = np.linalg.eig(R_sample)
            if i == 0 and snr_dB == -20:
                print()
                print("EIGENVALUES OF SAMPLE COVARIANCE MATRIX")
                print(w_sample[0])
                print(w_sample[1])
                print(w_sample[2])
                print(w_sample[3])

            # Sensor Selection Matrix (shape: (12,6))
            T = np.array([[1,0,0,0,0,0],
                          [1,0,0,0,0,0],
                          [0,1,0,0,0,0],
                          [0,1,0,0,0,0],
                          [0,0,1,0,0,0],
                          [0,0,1,0,0,0],
                          [0,0,0,1,0,0],
                          [0,0,0,1,0,0],
                          [0,0,0,0,1,0],
                          [0,0,0,0,1,0],
                          [0,0,0,0,0,1],
                          [0,0,0,0,0,1]])

            # Push-Sum Matrix (shape: (6,6))
            P_push = np.array([[0.2,0.2,0.2,0  ,0  ,0],
                          [0.2,0.2,0.2,0  ,0  ,0],
                          [0.6,0.6,0.2,0.2,0  ,0],
                          [0  ,0  ,0.4,0.2,0.2,0.2],
                          [0  ,0  ,0  ,0.2,0.2,0.2],
                          [0  ,0  ,0  ,0.4,0.6,0.6]])

            # Average-Consensus Matrix (shape: (6,6))
            P_ave = np.array([[0.5,0.25,0.25,0  ,0  ,0],
                          [0.25,0.5,0.25,0  ,0  ,0],
                          [0.25,0.25,0.25,0.25,0  ,0],
                          [0  ,0  ,0.25,0.25,0.25,0.25],
                          [0  ,0  ,0  ,0.25,0.5,0.25],
                          [0  ,0  ,0  ,0.25,0.25,0.5]])

            # Weight Vector  (shape: (6,1))
            w = np.atleast_2d([1,1,1,1,1,1]).T

            if x == 0 or x == 1 or x == 2:
                
                if x == 0:
                    iteration = 10
                elif x == 1:
                    iteration = 20
                elif x == 2:
                    iteration = 30
            
                # Push-Sum Covariance Matrix Estimation
                R_push_numerator = np.multiply(T.dot(np.linalg.matrix_power(P_push,iteration)).dot(T.T), R_sample)
                R_push_denominator = T.dot(np.linalg.matrix_power(P_push,iteration)).dot(w).dot(np.ones((1,6))).dot(T.T)

                # Push Sum Covariance Matrix (shape: (12,12))
                R_push = K*np.multiply(R_push_numerator, (1/(R_push_denominator)))
                R = R_push

            if x == 3 or x == 4 or x == 5:
                
                if x == 0:
                    iteration = 10
                elif x == 1:
                    iteration = 20
                elif x == 2:
                    iteration = 30
                
                # Average Consensus Covariance Matrix Estimation      
                R_ave_con = K * np.multiply(T.dot(np.linalg.matrix_power(P_ave,iteration)).dot(T.T), R_sample)
                R = R_ave_con

            if x == 6 or x == 7 or x == 8:
                # Conventional ESPRIT Algorithm      
                R = R_sample               

    #         if i == 0 and snr_dB == -20:
    #             print()
    #             print("                            R_push")
    #             print("****************************************************************")
    #             print("****************************************************************")
    #             print(R_push)
    #             print()
    #             print("                           R_sample")
    #             print("****************************************************************")
    #             print("****************************************************************")
    #             print(R_sample)
    #             print()
    #             # Check Convergence Status
    #             print(R_push == R_sample)

            w_push, v_push = np.linalg.eig(R)

            # Upper group selection matrix J_up
            J_up = np.kron(np.eye(6),np.array([1,0]))

            # Lower group selection matrix J_down
            J_down = np.kron(np.eye(6),np.array([0,1]))

            # Push-Sum estimated signal eigenvector matrices
            U_s_push = v_push[:,:3]

            # Upper signal eigenvectors
            U_s_up = J_up.dot(U_s_push)

            # Lower signal eigenvectors
            U_s_down = J_down.dot(U_s_push)

            # Matrix including knowledge about DOAs of the source signals
            psi = np.linalg.inv((U_s_up.conj().T).dot(U_s_up)).dot((U_s_up.conj().T)).dot(U_s_down)

            w2, v2 = np.linalg.eig(psi)
            doa = []
            doa.append(np.arcsin(np.angle(w2[0])/np.pi)/(2*np.pi)*360)
            doa.append(np.arcsin(np.angle(w2[1])/np.pi)/(2*np.pi)*360)
            doa.append(np.arcsin(np.angle(w2[2])/np.pi)/(2*np.pi)*360)

            if i == 0:
                print()
                print("  DOAs of the source signals in degrees with SNR: " + str(snr_dB) )
                print("  DOAs of the source signals in degrees with N_samples: " + str(N_samples) )
                print("****************************************************************")
                print("****************************************************************")
                print("DOA of the first source signal:   " + str(doa[0]))
                print("DOA of the second source signal:   " + str(doa[1]))
                print("DOA of the third source signal:   " + str(doa[2]))

            # Derivation matrix
            D = np.vstack((np.gradient(A[:,0]), np.gradient(A[:,1]), np.gradient(A[:,2]))).T
            # Orthogonal complement of the projection matrix
            P_A = np.identity(12) - A.dot(np.linalg.inv(A.conj().T.dot(A))).dot(A.conj().T)
            # Emitter covariance matrix
            S = signal.dot(signal.conj().T)

            diff_1 = min(abs(doa[0]-angles*360/(2*np.pi)))
            diff_2 = min(abs(doa[1]-angles*360/(2*np.pi)))
            diff_3 = min(abs(doa[2]-angles*360/(2*np.pi)))

            # Here in every iteration doa angles change and we take 1/500 times of it. This corresponds to Monte-Carlo simulation.
            MSE[snr_dB+20] = MSE[snr_dB+20] + 1/3*1/500*((diff_1)**2+(diff_2)**2+(diff_3)**2)
            # MSE[N_samples-2] = MSE[N_samples-2] + 1/3*1/500*((diff_1)**2+(diff_2)**2+(diff_3)**2)
            # cramer[N_samples-2] = cramer[N_samples-2] + 1/500*cramer_rao(A, signal, angles, locations)
            # cramer[snr_dB+20] = cramer[snr_dB+20] + 1/500*cramer_rao(A, signal, angles, locations)
            
            if i == 499: 
                #print("MSE")
                print(MSE[snr_dB+20])
                #print(MSE[N_samples-2])
                #print("Cramer Rao Bound")
                #print(cramer[N_samples-2])
                #print(cramer[snr_dB+20])
    
    RMSE = np.sqrt(MSE)
    
    # load the results in memory since it takes too much time to obtain it again.(1000 iteration for N=1000)
    # (Don't forget to change the name when you change some parameters above)
    if x == 0:
        with open("Push_Sum_ESPRIT_SNR_10dB_P_10.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 1:
        with open("Push_Sum_ESPRIT_SNR_10dB_P_20.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)    
    if x == 2:
        with open("Push_Sum_ESPRIT_SNR_10dB_P_30.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 3:
        with open("Average_Consensus_ESPRIT_SNR_10dB_P_10.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 4:
        with open("Average_Consensus_ESPRIT_SNR_10dB_P_20.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 5:
        with open("Average_Consensus_ESPRIT_SNR_10dB_P_30.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 6:
        with open("Conventional_ESPRIT_SNR_10dB.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 7:
        with open("Cramer_Rao_Bound_1000_Samples.txt", "wb") as fp:   #Pickling
            pickle.dump(cramer, fp)
    if x == 8:
        with open("Cramer_Rao_Bound_SNR_10dB.txt", "wb") as fp:   #Pickling
            pickle.dump(cramer, fp) 


SIGNAL TO NOISE RATIO
0.08617015970440657

EIGENVALUES OF SAMPLE COVARIANCE MATRIX
(212.27613236393395-5.8009748913697504e-15j)
(170.50795221044638+3.3941427949062453e-15j)
(52.645711232448754+3.2507975342023634e-15j)
(149.57872300204474-5.5659827995904195e-15j)

  DOAs of the source signals in degrees with SNR: -20
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   -8.33689453212301
DOA of the second source signal:   -65.97109382765576
DOA of the third source signal:   -6.793608672623361
477.90528025500544
SIGNAL TO NOISE RATIO
0.12052362029493859

  DOAs of the source signals in degrees with SNR: -19
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source sig

5.085250649190676
SIGNAL TO NOISE RATIO
0.7731232956646393

  DOAs of the source signals in degrees with SNR: -2
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   -16.165900638483063
DOA of the second source signal:   0.5123704164714353
DOA of the third source signal:   -7.709307663401443
4.049068973994469
SIGNAL TO NOISE RATIO
0.8472564119037495

  DOAs of the source signals in degrees with SNR: -1
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.789291538954093
DOA of the second source signal:   -14.498354774146398
DOA of the third source signal:   -11.592765621761327
3.5474928754925035
SIGNAL TO NOISE RATIO
1.0778366724653579

  DOAs of t

1.6211970892719176
SIGNAL TO NOISE RATIO
6.822793231074151

  DOAs of the source signals in degrees with SNR: 16
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   5.323083840922374
DOA of the second source signal:   -11.224639040445785
DOA of the third source signal:   -8.253180642032202
1.5686387581264516
SIGNAL TO NOISE RATIO
6.452699181018096

  DOAs of the source signals in degrees with SNR: 17
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   6.1417194853351145
DOA of the second source signal:   -12.05835826622789
DOA of the third source signal:   -8.003935131074279
1.6036954838351627
SIGNAL TO NOISE RATIO
8.823266581271993

  DOAs of the 

1.5863809639387285
SIGNAL TO NOISE RATIO
54.0250308406137

  DOAs of the source signals in degrees with SNR: 34
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.525747767787675
DOA of the second source signal:   -12.305110429637335
DOA of the third source signal:   -9.541228398060875
1.556329382966918
SIGNAL TO NOISE RATIO
54.41026187951624

  DOAs of the source signals in degrees with SNR: 35
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.535289274096285
DOA of the second source signal:   -12.473291146964788
DOA of the third source signal:   -9.443270166292189
1.547879123147035
SIGNAL TO NOISE RATIO
63.80003197500837

  DOAs of the sou

1.5603316848957323
SIGNAL TO NOISE RATIO
364.9879088656984

  DOAs of the source signals in degrees with SNR: 52
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.758712246546071
DOA of the second source signal:   -12.607152149188927
DOA of the third source signal:   -9.592644461966442
1.622253843823335
SIGNAL TO NOISE RATIO
423.65430405744917

  DOAs of the source signals in degrees with SNR: 53
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.853520162929975
DOA of the second source signal:   -12.2399512703646
DOA of the third source signal:   -9.19182093543784
1.5478746023034011
SIGNAL TO NOISE RATIO
513.47720810685

  DOAs of the sourc